In [1]:
import pandas as pd
import numpy as np


In [27]:

# IMPORT DATA
loyalty_df = pd.read_csv("../data/receipt_item-2025010915-fe8603095238452290775091f1eabc2f.csv", skiprows=1)
transctional_df=pd.read_csv('../data/2023 to 2024 Corp by stores ticket level.csv')

# LOYALTY PREPROCESS


datetime_col=['Receipt DateTime']
float_col=['Menu Item ID','Quantity','Rate','Transaction Number',
'Bar Code','Guest ID','Location ID','Checkin ID','Revenue ID',
'Store Number']

loyalty_df[datetime_col] = loyalty_df[datetime_col].apply(pd.to_datetime, errors='coerce')
loyalty_df[float_col] =loyalty_df[float_col] .apply(pd.to_numeric, errors='coerce')
loyalty_df['date']=loyalty_df['Receipt DateTime'].dt.date
# loyalty_df['time']=loyalty_df['Receipt DateTime'].dt.time


loyalty_df=loyalty_df[loyalty_df['Location Name']!='(CLOSED) Windsor (Royal Vista Cir)']

loyalty_df['StoreId'] = loyalty_df['Location Name'].str[-4:]
loyalty_df['StoreId']=loyalty_df['StoreId'].astype(float)

loyalty_df['time'] = np.where(loyalty_df['StoreId'].isin([1051,1052]),loyalty_df['Receipt DateTime'] + pd.Timedelta(hours=1),loyalty_df['Receipt DateTime'])
loyalty_df['time']=pd.to_datetime(loyalty_df['time'])
loyalty_df['hour']=loyalty_df['time'].dt.hour
loyalty_df['time'] = loyalty_df['time'].dt.time
loyalty_df['month']=loyalty_df['Receipt DateTime'].dt.month
loyalty_df['year']=loyalty_df['Receipt DateTime'].dt.year


loyalty_df=loyalty_df[loyalty_df['Menu Item Type']=='M']
store_ids=loyalty_df['StoreId'].unique()
print(loyalty_df['StoreId'].unique())
loyalty_df=loyalty_df[loyalty_df['StoreId'].isin([1001, 1003, 1005, 1006, 1008, 1051,1052])]
print(loyalty_df['StoreId'].unique())

# # Concatenate 'StoreID', 'Date', and 'CheckTime' to create 'Key'
loyalty_df['Key'] = loyalty_df['StoreId'].astype(str) + "_" + loyalty_df['date'].astype(str) + "_" + loyalty_df['time'].astype(str)

# MAKING SUMMARY TABLE FROM LOYALTY TABLE
summary_loyal=loyalty_df.groupby(['Key', 'Checkin ID'])['Menu Item ID'].unique().reset_index()
# TRANSACTIONAL DATA PRE_PROCESS

datetime_col=['Date','CheckTime']
float_col=['StoreId','UniqueCheckId','MenuItemId','PLU']



transctional_df[datetime_col] = transctional_df[datetime_col].apply(pd.to_datetime, errors='coerce')
transctional_df[float_col] =transctional_df[float_col] .apply(pd.to_numeric, errors='coerce')

transctional_df['date']=transctional_df['Date'].dt.date
# transctional_df['time']=transctional_df['CheckTime'].dt.time


# Add one hour
transctional_df['time'] = transctional_df['CheckTime'] + pd.Timedelta(hours=1)
transctional_df['hour']=transctional_df['Date'].dt.hour
transctional_df['month']=transctional_df['Date'].dt.month
transctional_df['year']=transctional_df['Date'].dt.year
transctional_df['time'] = transctional_df['CheckTime'].dt.time

# Concatenate 'StoreID', 'Date', and 'CheckTime' to create 'Key'
transctional_df['Key'] = transctional_df['StoreId'].astype(str) + "_" + transctional_df['date'].astype(str) + "_" + transctional_df['time'].astype(str)

summary_trans=transctional_df.groupby(['Key', 'UniqueCheckId'])['PLU'].unique().reset_index()


/var/folders/lg/b8pw7x296y397cn0g302yt680000gp/T/ipykernel_839/3936155120.py:2: DtypeWarning: Columns (1,10,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  loyalty_df = pd.read_csv("../data/receipt_item-2025010915-fe8603095238452290775091f1eabc2f.csv", skiprows=1)
/var/folders/lg/b8pw7x296y397cn0g302yt680000gp/T/ipykernel_839/3936155120.py:3: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  transctional_df=pd.read_csv('../data/2023 to 2024 Corp by stores ticket level.csv')
/var/folders/lg/b8pw7x296y397cn0g302yt680000gp/T/ipykernel_839/3936155120.py:13: FutureWarning: Parsed string "January 01, 2023 09:58 AM MST" included an un-recognized timezone "MST". Dropping unrecognized timezones is deprecated; in a future version this will raise. Instead pass the string without the timezone, then use .tz_localize to convert to a recognized timezone.
  loyalty_df[datetime_col] = loyalty_df[datetime_col].app

[1008. 1031. 1006. 1003. 1052. 1005. 2609. 1001. 2620. 1036. 1051. 2616.]
[1008. 1006. 1003. 1052. 1005. 1001. 1051.]


/var/folders/lg/b8pw7x296y397cn0g302yt680000gp/T/ipykernel_839/3936155120.py:50: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  transctional_df[datetime_col] = transctional_df[datetime_col].apply(pd.to_datetime, errors='coerce')


In [49]:
import pandas as pd

# Sample DataFrames
df1 = summary_loyal.copy()
df2 = summary_trans.copy()

df1.rename(columns={'Menu Item ID':'list_of_items'},inplace=True)

df2.rename(columns={'PLU':'list_of_items'},inplace=True)
df1.columns


Index(['Key', 'Checkin ID', 'list_of_items'], dtype='object')

In [50]:
import pandas as pd

# Sample DataFrames

# df1=df1[df1['Key']=='1005.0_2024-04-10_13:35:00']
# df2=df2[df2['Key']=='1005.0_2024-04-10_13:35:00']
# Convert list_of_items to sets for subset checking
df1['list_of_items_set'] = df1['list_of_items'].apply(set)
df2['list_of_items_set'] = df2['list_of_items'].apply(set)

# Function to check if any key-list combination from df1 is a subset in df2
def is_subset(row):
    for _, df1_row in df1[df1['Key'] == row['Key']].iterrows():
        if df1_row['list_of_items_set'].issubset(row['list_of_items_set']):
            return 'yes'
    return 'no'

# Apply function to df2
df2['member'] = df2.apply(is_subset, axis=1)

# Drop helper column
df2.drop(columns=['list_of_items_set'], inplace=True)

# Display result
df2.shape


KeyboardInterrupt: 

In [48]:
df2



,Key,UniqueCheckId,list_of_items,member
669082,1005.0_2024-04-10_13:35:00,10124010052040,"[58843.0, 59388.0]",no
669083,1005.0_2024-04-10_13:35:00,10124010053191,"[52075.0, 51893.0]",yes


In [ ]:
df2[df2['Key']=='1005.0_2024-04-10_13:35:00']


In [3]:
A = set(loyalty_df['Key'].unique())
B = set(transctional_df['Key'].unique())

# Compute the difference
difference = list(A - B)



In [4]:
processed_df= pd.read_csv("output1.csv") 


/var/folders/lg/b8pw7x296y397cn0g302yt680000gp/T/ipykernel_839/233877256.py:1: DtypeWarning: Columns (0,3,4,5,7,8,9,10,13,17,18,21,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  processed_df= pd.read_csv("output1.csv")


In [5]:
transctional_df["Key"].nunique()
transctional_df.shape


(8448883, 25)

In [6]:
processed_df["Key"].nunique()
processed_df.shape


(307173, 26)

In [7]:
df=transctional_df[transctional_df['Key'].isin(list(processed_df['Key']))]
df["Key"].nunique()
df['loyal_member']="yes"
df.shape


/var/folders/lg/b8pw7x296y397cn0g302yt680000gp/T/ipykernel_839/4019314980.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['loyal_member']="yes"


(385336, 26)

In [8]:
df1=transctional_df[~transctional_df['Key'].isin(list(processed_df['Key']))]
df1["Key"].nunique()
df1['loyal_member']="no"
df1.shape


/var/folders/lg/b8pw7x296y397cn0g302yt680000gp/T/ipykernel_839/3256302345.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['loyal_member']="no"


(8063547, 26)

In [9]:
8063547+385336


8448883

In [10]:

result = pd.concat([df, df1])
result.shape


(8448883, 26)

In [11]:
len(difference)


10556

In [12]:
result.head()


,StoreId,Date,CheckTime,UniqueCheckId,PLU,MenuItemId,MenuItemName,ItemCount,Amount,Tax,...,Order_type,discount_amt,DiscountName,date,time,hour,month,year,Key,loyal_member
111,1006.0,2023-01-05,2025-02-12 09:31:00,523010063,49760.0,843.0,Remove Potato Oles,1.0,0.00,0.00,...,Dine,NaN,NaN,2023-01-05,09:31:00,0.0,1.0,2023.0,1006.0_2023-01-05_09:31:00,yes
112,1006.0,2023-01-05,2025-02-12 09:31:00,523010063,49762.0,19.0,Meat & Potato Breakfast Burrito - Sausage,1.0,3.79,0.23,...,Dine,NaN,NaN,2023-01-05,09:31:00,0.0,1.0,2023.0,1006.0_2023-01-05_09:31:00,yes
113,1006.0,2023-01-05,2025-02-12 09:31:00,523010063,50254.0,10.0,Junior Breakfast Burrito - Sausage,1.0,1.69,0.10,...,Dine,NaN,NaN,2023-01-05,09:31:00,0.0,1.0,2023.0,1006.0_2023-01-05_09:31:00,yes
281,1005.0,2023-02-04,2025-02-12 10:17:00,3523010052,49762.0,19.0,Meat & Potato Breakfast Burrito - Sausage,1.0,3.79,0.18,...,Take,NaN,NaN,2023-02-04,10:17:00,0.0,2.0,2023.0,1005.0_2023-02-04_10:17:00,yes
282,1005.0,2023-02-04,2025-02-12 10:17:00,3523010052,51893.0,384.0,"Large, Drink",1.0,2.69,0.12,...,Take,NaN,NaN,2023-02-04,10:17:00,0.0,2.0,2023.0,1005.0_2023-02-04_10:17:00,yes


In [13]:
# Reduction of datasets
#filter keys in loyalty tble that are present in transactionl df, this way we reduce search for non existed keys.=list(set(receipt_item_summary['Key'].unique()) & set(transctional_df['Key'].unique()))
#step1: find intersaction of keys bwtween transaction and loyal
loyalty_keys=list(set(loyalty_df['Key'].unique()) & set(transctional_df['Key'].unique()))

# #step 2: keep only keys that are in transation
summary_loyal_filtered=loyalty_df[loyalty_df['Key'].isin(loyalty_keys)]

# step 3: check the unique keys
print('initil no of keys',loyalty_df['Key'].nunique())
print('number of common keys',summary_loyal_filtered['Key'].nunique())

# print('within common keys, no of unique Checkin ID in loyalty table are:', summary_loyal['Checkin ID'].nunique())
# print('within common keys, no of unique UniqueCheckId in transactional table are:', transctional_df[transctional_df['Key'].isin(loyalty_keys)]['UniqueCheckId'].nunique())


print('keys found in transactional',result[result['loyal_member']=='yes']['Key'].nunique())
print('keys found in transactional',((result[result['loyal_member']=='yes']['Key'].nunique())/83017)*100)


initil no of keys 93573
number of common keys 83017
keys found in transactional 62617
keys found in transactional 75.42671982846886


In [14]:
# set(result[result['loyal_member']=='yes']['Key'].unique()) - set(summary_loyal_filtered['Key'].unique())



In [15]:
# result.to_csv('result.csv')


In [16]:
import csv
with open('result.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(result.columns)  # Write header
    for index, row in result.iterrows():
        writer.writerow(row.astype(str))  # Convert row to string before writing


In [17]:
d=pd.read_csv('result.csv')


/var/folders/lg/b8pw7x296y397cn0g302yt680000gp/T/ipykernel_839/1668785167.py:1: DtypeWarning: Columns (14,18) have mixed types. Specify dtype option on import or set low_memory=False.
  d=pd.read_csv('result.csv')


In [18]:
d['loyal_member'].unique()


array(['yes', 'no'], dtype=object)

In [38]:
result[result['loyal_member']=='yes']['UniqueCheckId'].nunique()


80375

In [20]:

summary_processed=result.groupby(['Key', 'UniqueCheckId','loyal_member'])['PLU'].unique().reset_index()
summary_processed


,Key,UniqueCheckId,loyal_member,PLU
0,1001.0_2023-01-01_09:57:00,123010012000,no,"[51892.0, 51891.0, 49098.0, 50420.0]"
1,1001.0_2023-01-01_10:01:00,123010014000,no,"[50064.0, 47478.0]"
2,1001.0_2023-01-01_10:02:00,123010014001,no,"[49016.0, 49762.0, 47478.0, 49761.0, 50064.0]"
3,1001.0_2023-01-01_10:04:00,123010014002,no,"[51892.0, 47478.0, 52071.0]"
4,1001.0_2023-01-01_10:05:00,123010013000,no,"[52061.0, 51891.0, 50396.0, 49083.0, 52588.0]"
...,...,...,...,...
1817388,nan_NaT_NaT,3662401052140329385,no,[nan]
1817389,nan_NaT_NaT,3662401052160209971,no,[nan]
1817390,nan_NaT_NaT,3662401052174231602,no,[nan]
1817391,nan_NaT_NaT,3662401052175922941,no,[nan]


In [33]:
summary_processed[summary_processed['Key']=='1005.0_2024-04-10_13:35:00']




,Key,UniqueCheckId,loyal_member,PLU
669082,1005.0_2024-04-10_13:35:00,10124010052040,yes,"[58843.0, 59388.0]"
669083,1005.0_2024-04-10_13:35:00,10124010053191,yes,"[52075.0, 51893.0]"


In [34]:

summary_trans[summary_trans['Key']=='1005.0_2024-04-10_13:35:00']


,Key,UniqueCheckId,PLU
669082,1005.0_2024-04-10_13:35:00,10124010052040,"[58843.0, 59388.0]"
669083,1005.0_2024-04-10_13:35:00,10124010053191,"[52075.0, 51893.0]"


In [35]:

summary_loyal[summary_loyal['Key']=='1005.0_2024-04-10_13:35:00']



,Key,Checkin ID,Menu Item ID
39942,1005.0_2024-04-10_13:35:00,1358440598,"[51893.0, 52075.0]"


In [30]:
loyalty_df[loyalty_df['Key']=='1001.0_2023-01-01_10:05:00']


,Transaction Number,Bar Code,Guest ID,Location ID,Checkin ID,Receipt DateTime,Revenue ID,Revenue Code,Location Name,Store Number,...,Menu Item Group,Quantity,Rate,date,StoreId,time,hour,month,year,Key


In [28]:

transctional_df[transctional_df['Key']== '1006.0_2023-01-05_09:31:00'].groupby(['Key', 'UniqueCheckId'])['PLU'].unique().reset_index()



,Key,UniqueCheckId,PLU
0,1006.0_2023-01-05_09:31:00,523010063,"[49760.0, 49762.0, 50254.0]"
1,1006.0_2023-01-05_09:31:00,523010063067,"[51892.0, 49204.0, 47478.0, 52065.0]"
